In [3]:
from commom.database.queries.query_lojas import QUERY_LOJAS
from commom.database.queries.query_vendas_nf_novo import QUERY_VENDAS_NF_NOVO
from commom.database.queries.query_metas import QUERY_METAS
from commom.database.queries.query_parcelas import QUERY_PARCELAS
from commom.database.queries.query_vendas_nf import QUERY_VENDAS_NF
from commom.database.queries.query_vendedores import QUERY_VENDEDORES
from commom.kpi_data.kpi_data_manager import DataHandler, KpiDataManager
from typing import Tuple
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta
from innovation_messenger import Messenger
import os
from innovation_messenger import IEmailProperties

In [4]:
manager = KpiDataManager()
handler = DataHandler()

In [5]:
df_fato_vendas = handler.read_from_local_pickle('fato_vendas.pkl')
df_lojas = handler.read_from_local_pickle('df_lojas.pkl')
df_vendedores = handler.read_from_local_pickle('df_vendedores.pkl')
df_vendas = handler.read_from_local_pickle('df_vendas.pkl')
df_parcelas = handler.read_from_local_pickle('df_parcelas.pkl')
df_metas = handler.read_from_local_pickle('df_metas.pkl')

# df = handler.read_from_bigquery(QUERY_VENDAS_NF_NOVO)
# df_lojas = handler.read_from_bigquery(QUERY_LOJAS)
# df_vendedores = handler.read_from_bigquery(QUERY_VENDEDORES)
# df_vendas = handler.read_from_bigquery(QUERY_VENDAS_NF)
# df_parcelas = handler.read_from_bigquery(QUERY_PARCELAS)
# df_metas = handler.read_from_bigquery(QUERY_METAS).rename(
#         columns={"cpf_vendedor": "cpf_vendedor_inteiro", "loja": "distributorId"}
#     )


In [7]:
def _abreviate_vendedor_name(full_name: str) -> str:
        full_name = full_name.replace("   ", " ")
        full_name = full_name.replace("  ", " ")

        if len(full_name.title().split(" ")) > 1:
            splited_name = full_name.title().split(" ")
            nome_completo = splited_name[0]
            splited_name.pop(0)

            for name in splited_name:
                nome_completo += " " + name[0] + "."

            return nome_completo

        return full_name

In [6]:
ids_loja = [
        "2560",
        "2510",
        "2540",
        "0595",
        "0529",
        "0768",
        "0749",
        "0760",
        "2559",
        "0753",
        "0662",
        "2585",
        "2511",
        "2522",
        "2574",
        "0571",
        "0743",
        "0713",
        "2565",
        "2572",
        "0775",
        "2557",
        "2558",
        "0647",
        "0765",
        "1071",
        "10C5",
        "1010",
        "10F3",
        "1012",
        "0706",
        "0742",
        "2512",
        "2536",
        "0641"
    ]

# Fluxo antigo

In [8]:
# Service
## -> inicio
###     -> fetch data from bq
df_fato_vendas = handler.read_from_local_pickle('fato_vendas.pkl')
df_lojas = handler.read_from_local_pickle('df_lojas.pkl')
df_vendedores = handler.read_from_local_pickle('df_vendedores.pkl')
df_vendas = handler.read_from_local_pickle('df_vendas.pkl')
df_parcelas = handler.read_from_local_pickle('df_parcelas.pkl')
df_metas = handler.read_from_local_pickle('df_metas.pkl')

In [10]:
###     -> _build_df_vendas_nf_pdv
df_vendas_temp = df_vendas.copy()

PDV_TYPE_LIST = ["STORE", "PI - 360", "PI", "EP", "REFUND"]

mask_pdv = df_vendas_temp["tipo_transacao"].isin(PDV_TYPE_LIST)

df_vendas_pdv = df_vendas_temp[mask_pdv]
df_vendas_pdv.loc[:, "tipo_transacao"] = df_vendas_pdv["tipo_transacao"].replace("PI - 360", "STORE")
df_vendas_pdv.loc[:, "tipo_transacao"] = df_vendas_pdv["tipo_transacao"].replace("PI", "STORE")
df_vendas_pdv.loc[:, "tipo_transacao"] = df_vendas_pdv["tipo_transacao"].replace("EP", "STORE")

df_vendas_pdv = df_vendas_pdv.reset_index(drop=True)
df_vendas_pdv.sample(2)

,distributorId,year,month,parent_orderId,displaycode,invoice_date,tipo_transacao,cpf_vendedor_inteiro,amount,paidFreight,customerShippingCost,grossValue,discount_value,store_cost,net_value
358847,2530,2024,8,ff702a6e-fbc0-4a23-abea-fbba48406294,ff702a6e-fbc0-4a23-abea-fbba48406294,2024-08-05,STORE,36191293801,1,0.0,0.0,199.9,0.00,81.98,199.90
473777,10H2,2024,11,a81363fa-0cbf-4689-bafc-4a3db3e66bf1,DFDGLV8W4YG,2024-11-02,STORE,49801593890,1,31.0,0.0,179.9,17.99,40.97,161.91


In [11]:
###     -> _build_df_parcelas_with_displaycode(self):
df_vendedor_displaycode = (
    df_vendas_pdv.groupby(["displaycode", "cpf_vendedor_inteiro", "distributorId"])["year"]
        .count()
        .reset_index()
        .drop(columns="year")
)

df_parcelas_with_displaycode = df_parcelas.merge(df_vendedor_displaycode, how="left", on="displaycode")
df_parcelas_with_displaycode.sample(2)

,year,month,data_emissao_NF,displaycode,n_parcelas,cpf_vendedor_inteiro,distributorId
71635,2024,3,2024-03-21,6ecb9b3c-20df-478e-939c-ba8c3fec3ab5,1.0,10436542838,10J9
304926,2024,12,2024-12-24,6f94c775-06f0-4b07-9dfb-7a664507cfdb,2.0,NaN,NaN


In [12]:
###     -> atualiza o df_nome_vendedor com os dados de nome e cpf da tabela df_vendedores
df_nome_vendedor = df_vendedores[["name", "cpf"]]
df_nome_vendedor.columns = ["vendedor", "cpf_vendedor_inteiro"]
df_nome_vendedor.loc[:, "vendedor"] = df_nome_vendedor["vendedor"].apply(_abreviate_vendedor_name)
df_nome_vendedor.sample(3)

,vendedor,cpf_vendedor_inteiro
3092,Renata D.,27035976817
1090,Jênifer A.,09359239500
1961,Larissa R. A.,35298872859


In [13]:
###     -> self date
yesterday_date = datetime.datetime(2024,10,6)
yesterday_date

datetime.datetime(2024, 10, 6, 0, 0)

In [14]:
###     -> _build_masks
def build_masks(kpi_type, kpi_period):
    mask_tickets = pd.Series(np.ones(df_vendas_pdv.shape[0]).astype(bool))
    mask_vendas = pd.Series(np.ones(df_vendas_pdv.shape[0]).astype(bool))
    mask_parcelas = pd.Series(np.ones(df_parcelas_with_displaycode.shape[0]).astype(bool))

    if kpi_type == "loja":
        mask_meta = df_metas["tipo_meta"] == "Meta Loja PDV"
    elif kpi_type == "vendedor":
        mask_meta = df_metas["tipo_meta"] == "Meta Vendedor PDV"

    if kpi_period == "Dia":
        mask_yesterday_vendas = df_vendas_pdv["invoice_date"] == yesterday_date
        mask_vendas = mask_yesterday_vendas
        mask_tickets = mask_tickets & mask_yesterday_vendas

        mask_metas_yesterday = df_metas["date"] == yesterday_date
        mask_meta = mask_meta & mask_metas_yesterday

        mask_parcelas = df_parcelas_with_displaycode["data_emissao_NF"] == yesterday_date

    return mask_tickets, mask_vendas, mask_parcelas, mask_meta

In [15]:
###     -> _build_kpis_dataframe -> _merge_dataframes
def _build_kpis_dataframe(kpi_type, kpi_period, mask_vendas,mask_tickets,mask_parcelas,mask_meta):
    # yesterday_date = datetime.datetime(2025,1,6)
    AGG_DICT = {"discount_value": "sum", "net_value": "sum", "store_cost": "sum"}

    GROUP_BY_COLUMNS_DICT = {
        "Ano": ["distributorId", "year"],
        "Mês": ["distributorId", "year", "month"],
        "Dia": ["distributorId", "year", "month"],
    }

    group_by_columns = GROUP_BY_COLUMNS_DICT[kpi_period]
    if kpi_type == "vendedor":
        group_by_columns = ["cpf_vendedor_inteiro"] + group_by_columns

    df_kpis: pd.DataFrame = df_vendas_pdv[mask_vendas].groupby(group_by_columns).agg(AGG_DICT).reset_index()

    df_kpis = df_kpis.merge(
        df_parcelas_with_displaycode[mask_parcelas].groupby(group_by_columns)["n_parcelas"].mean().reset_index(),
            on=group_by_columns,
            how="left",
        )

    df_kpis = df_kpis.merge(
        df_vendas_pdv[mask_tickets].groupby(group_by_columns)["parent_orderId"].nunique().reset_index().rename(columns={"parent_orderId": "tkts"}),
        how="left",
        )

    df_kpis = df_kpis.merge(
        df_vendas_pdv[mask_tickets].groupby(group_by_columns)["amount"].sum().reset_index(), 
        how="left"
    )

    df_kpis = df_kpis.merge(
        df_metas[mask_meta].groupby(group_by_columns)["meta"].sum().reset_index(), 
        how="left"
    )

    if kpi_type == "vendedor":
        df_kpis = df_kpis.merge(
        df_vendas_pdv.groupby(group_by_columns)["invoice_date"]
        .nunique()
        .reset_index()
        .rename(columns={"invoice_date": "days_of_work"}),
        how="left",
        )

    if kpi_period == "Dia":
        # yesterday_date = datetime.datetime.today() - datetime.timedelta(days=1)
        df_kpis["Dia"] = yesterday_date.day

    if kpi_type == "loja":
        if kpi_period == "Ano":
            df_kpis["tkts_per_day"] = df_kpis["tkts"] / yesterday_date.timetuple().tm_yday
        if kpi_period == "Mês":
            df_kpis["tkts_per_day"] = df_kpis["tkts"] / yesterday_date.day
        if kpi_period == "Dia":
            df_kpis["tkts_per_day"] = df_kpis["tkts"]

    elif kpi_type == "vendedor":
        if kpi_period == "Mês":
            df_kpis["tkts_per_day"] = df_kpis["tkts"] / df_kpis["days_of_work"].round(2)
        else:
            df_kpis["tkts_per_day"] = df_kpis["tkts"]

    df_kpis["type"] = kpi_period

    df_kpis["mkp"] = (df_kpis["net_value"] / df_kpis["store_cost"]).round(2)
    df_kpis["tkt_medio"] = df_kpis["net_value"] / df_kpis["tkts"].round(2)
    df_kpis["meta_percentual"] = (df_kpis["net_value"] / df_kpis["meta"] * 100).round(2)
    df_kpis["PA"] = df_kpis["amount"] / df_kpis["tkts"].round(2)
    df_kpis["discount_percentual"] = ((df_kpis["discount_value"] / df_kpis["net_value"]) * 100).round(2)
    
    return df_kpis

In [16]:
###     -> cria um dicionário de dataframes kpis para cada período para as lojas
###     -> cria os dataframes de kpi para vendedor e para lojas

kpis_loja_dict = {}
for kpi_period in ["Dia", "Mês", "Ano"]:
    mask_vendas,mask_tickets,mask_parcelas,mask_meta = build_masks(kpi_period=kpi_period, kpi_type='loja')
    kpis_loja_dict[f"kpis_loja_{kpi_period.lower()}"] = _build_kpis_dataframe(
                kpi_type="loja", 
                kpi_period=kpi_period,
                mask_vendas=mask_vendas,
                mask_tickets=mask_tickets,
                mask_parcelas=mask_parcelas,
                mask_meta=mask_meta
            )


kpis_vendedor_dict = {}
for kpi_period in ["Dia", "Mês", "Ano"]:
    mask_vendas,mask_tickets,mask_parcelas,mask_meta = build_masks(kpi_period=kpi_period, kpi_type='vendedor')
    kpis_vendedor_dict[f"kpis_vendedor_{kpi_period.lower()}"] = _build_kpis_dataframe(
        kpi_type="vendedor", 
        kpi_period=kpi_period,
        mask_vendas=mask_vendas,
        mask_tickets=mask_tickets,
        mask_parcelas=mask_parcelas,
        mask_meta=mask_meta
    )

df_kpis_loja = pd.concat(list(kpis_loja_dict.values()))
df_kpis_vendedor = pd.concat(list(kpis_vendedor_dict.values()))
df_kpis_vendedor = df_kpis_vendedor.dropna(subset="cpf_vendedor_inteiro")
mask = df_kpis_vendedor["cpf_vendedor_inteiro"] != "Devolução"
df_kpis_vendedor = df_kpis_vendedor[mask]

In [17]:
###     -> format all tables
#######    -> format store kpis
def format_store_kpis(id_loja):
    mask = df_kpis_loja["distributorId"] == id_loja

    df_store = df_kpis_loja[mask].fillna(0).copy()
    df_store_current_month = df_store.loc[df_store.groupby(["type"])["month"].idxmax()].drop_duplicates()

    rename_dict = {
        "net_value": "Venda Loja (PDV)",
        "meta": "Meta Loja (PDV)",
        "meta_percentual": "Meta % (PDV)",
        "mkp": "Markup",
        "tkts_per_day": "Tickets/Dia",
        "PA": "Peças por Atendimento",
        "tkt_medio": "Ticket Médio",
        # 'n_parcelas':'Parcela Média',
        "discount_percentual": "Desconto Médio %",
        "type": "Período",
    }

    format_money = "R$ {:,.0f}"
    format_percentage = "{:,.0f}%"

    format_dict = {
        "Venda Loja (PDV)": format_money,
        "Meta Loja (PDV)": format_money,
        "Meta % (PDV)": format_percentage,
        "Markup": "{:,.2f}",
        "Tickets/Dia": "{:,.1f}",
        "Peças por Atendimento": "{:,.1f}",
        "Ticket Médio": format_money,
        # 'Parcela Média':'{:,.1f}',
        "Desconto Médio %": format_percentage,
    }

    df_store_formated = df_store_current_month[rename_dict.keys()].round(2).rename(columns=rename_dict)

    for key, format in format_dict.items():
        df_store_formated[key] = df_store_formated[key].map(format.format)

        if format == format_money:
            df_store_formated[key] = df_store_formated[key].astype(str).str.replace(",", ".")

    try:
        df_store_formated = df_store_formated.set_index("Período").reset_index().T[[1, 2, 0]]
    except KeyError:
        df_store_formated = pd.DataFrame()
    
    return df_store_formated


In [18]:
###     -> format all tables
#######    -> format_vendedor_kpis_mes
def format_vendedor_kpis_mes(id_loja):
    mask = df_kpis_vendedor["distributorId"] == id_loja
    df_vendedor = df_kpis_vendedor[mask].reset_index(drop=True)
    df_vendedor["month"] = df_vendedor["month"].fillna(0)

    mask_type = df_vendedor["type"].isin(["Mês"])
    mask_mes = df_vendedor["month"] == yesterday_date.month
    mask_ano = df_vendedor["year"] == yesterday_date.year
    mask_completa = mask_type & mask_mes & mask_ano
    df_vendedor_mes: pd.DataFrame = df_vendedor[mask_completa].reset_index(drop=True)
    df_vendedor_mes["normalized_net_value"] = (
        df_vendedor_mes["net_value"] * 100 / round(df_vendedor_mes["net_value"].max(), 2)
    )
    df_vendedor_mes["net_value_share"] = (
        df_vendedor_mes["net_value"] * 100 / round(df_vendedor_mes["net_value"].max(), 2)
    )
    df_vendedor_mes = df_nome_vendedor.merge(df_vendedor_mes, on="cpf_vendedor_inteiro").drop(
        columns="cpf_vendedor_inteiro"
    )

    rename_dict = {
        "vendedor": "Vendedor",
        "days_of_work": "Dias Trabalhados",
        "net_value": "Acumulado no Mês",
        "normalized_net_value": "Homogeneidade",
        "meta": "Meta do Mês",
        "meta_percentual": "Meta %",
        "mkp": "Markup",
        "tkts_per_day": "Tickets/Dia",
        "PA": "Peças por Atendimento",
        "tkt_medio": "Ticket Médio",
        # 'n_parcelas':'Parcela Média',
        "discount_percentual": "Desconto Médio %",
        # 'type':'Período'
    }

    format_money = "R$ {:,.0f}"
    format_percentage = "{:,.0f}%"

    format_dict = {
        "Acumulado no Mês": format_money,
        "Homogeneidade": format_percentage,
        "Meta do Mês": format_money,
        "Meta %": format_percentage,
        "Markup": "{:,.2f}",
        "Tickets/Dia": "{:,.1f}",
        "Peças por Atendimento": "{:,.1f}",
        "Ticket Médio": format_money,
        # 'Parcela Média':'{:,.1f}',
        "Desconto Médio %": format_percentage,
    }

    df_vendedor_mes_formatado = (
        df_vendedor_mes[rename_dict.keys()]
        .sort_values("normalized_net_value", ascending=False)
        .round(2)
        .rename(columns=rename_dict)
    )

    for key, format in format_dict.items():
        df_vendedor_mes_formatado[key] = df_vendedor_mes_formatado[key].map(format.format)
        if format == format_money:
            df_vendedor_mes_formatado[key] = df_vendedor_mes_formatado[key].astype(str).str.replace(",", ".")

    df_vendedor_mes_formatado = df_vendedor_mes_formatado.reset_index(drop=True).T.reset_index().T

    return df_vendedor_mes_formatado.set_index(0)

In [19]:
###     -> format all tables
#######    -> format_vendedor_kpis_dia
def format_vendedor_kpis_dia(id_loja):
    mask = df_kpis_vendedor["distributorId"] == id_loja
    df_vendedor = df_kpis_vendedor[mask].reset_index(drop=True)
    df_vendedor["month"] = df_vendedor["month"].fillna(0)

    mask_type = df_vendedor["type"].isin(["Dia"])
    mask_mes = df_vendedor["month"] == yesterday_date.month
    mask_ano = df_vendedor["year"] == yesterday_date.year
    mask_dia = df_vendedor['Dia'] == yesterday_date.day
    df_vendedor_dia = df_vendedor[mask_dia & mask_mes & mask_ano & mask_type].reset_index(drop=True)
    
    df_vendedor_dia["normalized_net_value"] = (
        df_vendedor_dia["net_value"] * 100 / round(df_vendedor_dia["net_value"].max(), 2)
    )
    df_vendedor_dia["net_value_share"] = (
        df_vendedor_dia["net_value"] * 100 / round(df_vendedor_dia["net_value"].max(), 2)
    )
    df_vendedor_dia = df_nome_vendedor.merge(df_vendedor_dia, on="cpf_vendedor_inteiro").drop(
        columns="cpf_vendedor_inteiro"
    )

    rename_dict = {
        "vendedor": "Vendedor",
        "net_value": "Acumulado no Dia",
        "normalized_net_value": "Homogeneidade",
        "meta": "Meta do Dia",
        "meta_percentual": "Meta %",
        "mkp": "Markup",
        "tkts_per_day": "Tickets",
        "PA": "Peças por Atendimento",
        "tkt_medio": "Ticket Médio",
        # 'n_parcelas':'Parcela Média',
        "discount_percentual": "Desconto Médio %",
    }

    format_money = "R$ {:,.0f}"
    format_percentage = "{:,.0f}%"

    format_dict = {
        "Acumulado no Dia": format_money,
        "Homogeneidade": format_percentage,
        "Meta do Dia": format_money,
        "Meta %": format_percentage,
        "Markup": "{:,.2f}",
        "Tickets": "{:,.0f}",
        "Peças por Atendimento": "{:,.1f}",
        "Ticket Médio": format_money,
        # 'Parcela Média':'{:,.1f}',
        "Desconto Médio %": format_percentage,
    }

    df_vendedor_dia_formatado = (
        df_vendedor_dia[rename_dict.keys()]
        .sort_values("normalized_net_value", ascending=False)
        .rename(columns=rename_dict)
    )

    for key, format in format_dict.items():
        df_vendedor_dia_formatado[key] = df_vendedor_dia_formatado[key].map(format.format)
        if format == format_money:
            df_vendedor_dia_formatado[key] = df_vendedor_dia_formatado[key].astype(str).str.replace(",", ".")

    df_vendedor_dia_formatado = df_vendedor_dia_formatado.reset_index(drop=True).T.reset_index().T
    return df_vendedor_dia_formatado.set_index(0)

In [20]:
###     -> format all tables
#######    -> format_all_tables
def format_all_tables(id):
    df_store_formated = format_store_kpis(id)

    if df_store_formated.size == 0:
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

    df_vendedor_formated_month = format_vendedor_kpis_mes(id)

    df_vendedor_formated_day = format_vendedor_kpis_dia(id)

    return df_store_formated, df_vendedor_formated_month, df_vendedor_formated_day


In [51]:
import os
import smtplib
from dataclasses import dataclass
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from typing import Any, Callable, Dict, List, Optional, Union
from commom.logger import logger
from commom.base_classes.base_sender import BaseMessenger
from innovation_messenger.config import config


@dataclass
class IEmailProperties:
    subject: str
    body: str
    recipient: Union[str, List[str]]
    file_name: Optional[str] = None


class Enviador():
    def __init__(self) -> None:
        pass

    def send_email_teste(self, email_properties: IEmailProperties) -> bool:
        message = MIMEMultipart()
        message["Subject"] = email_properties.subject
        message["From"] = self.sender_email
        messageTo: str

        if isinstance(email_properties.recipient, List):
            messageTo = ", ".join(email_properties.recipient)
        else:
            messageTo = email_properties.recipient

        message["To"] = messageTo
        html_part = MIMEText(email_properties.body, 'html')
        message.attach(html_part)
        if isinstance(email_properties.recipient, List):
            messageTo = ", ".join(email_properties.recipient)
        else:
            messageTo = email_properties.recipient
            
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            try:
                server.login(user='inovacao@ammovarejo.com.br', password='inovaAM!05')
                server.sendmail('inovacao@ammovarejo.com.br', email_properties.recipient, message.as_string())
                return True
            except Exception as e:
                logger.error(e)
                return False

In [52]:
os.environ['EMAIL_PASSWORDS']="inovaAM!01,inovaAM!02,inovaAM!03,inovaAM!04,inovaAM!05,inovaAM!06"
os.environ['EMAIL_ACCOUNT'] = 'inovacao@ammovarejo.com.br'
# messenger = Messenger()
id = ids_loja[0]
loja, mes, dia = format_all_tables(id)

emailBody = ""

htmlValue = """
<html>
  <head></head>
  <body>
    {0}
  </body>
  <br/><br/>
</html>
""".format(loja.to_html())
emailBody += htmlValue

htmlValue = """
<html>
  <head></head>
  <body>
    {0}
  </body>
  <br/><br/>
</html>
""".format(dia.to_html())
emailBody += htmlValue

htmlValue = """
<html>
  <head></head>
  <body>
    {0}
  </body>
  <br/><br/>
</html>
""".format(mes.to_html())
emailBody += htmlValue

email_properties = IEmailProperties(
    subject='Teste - Base antiga',
    recipient=['joao.garcia@ammovarejo.com.br'],
    body=emailBody,
    # file_name=file_name,
)

Enviador().send_email_teste(
    email_properties=email_properties,
)


True

# Fluxo novo